In [10]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

digits = load_digits()
data   = np.asarray(digits.data, dtype='float32')
target = np.asarray(digits.target, dtype='int32')

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.15, random_state=37)

scaler  = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [39]:

#-------------------------------------------------------------
def one_hot(n_classes: int, y: int):
    # [y] - picks the row of the identity matrix, since thats
    #       the row that contains the diagonal value (one-hot-encoding)
    #       that we want.
    return np.eye(n_classes)[y]

#-------------------------------------------------------------
# SOFTMAX__MANUAL
def softmax(x):
    assert isinstance(x, np.ndarray)
    
    exp = np.exp(x)
    # print(np.sum(exp, axis=-1, keepdims=True))
    return exp / np.sum(exp, axis=-1, keepdims=True)

#-------------------------------------------------------------
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

#-------------------------------------------------------------
def get_activation_fn(p_activ_fn_str):
    if p_activ_fn_str == "sigmoid":
        return lambda p_z: sigmoid(p_z)

    elif p_activ_fn_str == "softmax":
        return lambda p_z: softmax(p_z)
            
#-------------------------------------------------------------


In [59]:
import numpy as np

#-------------------------------------------------------------
class Model:
    def __init__(self, p_layers_lst):
        self.layers_lst = p_layers_lst # :[:Layer]
        
class Layer:
    def __init__(self, p_size_M, p_size_N, p_activ_fn_str):
        self.W = np.random.uniform(size = (p_size_M, p_size_N), high=0.1, low=-0.1)
        self.b = np.random.uniform(size = p_size_N,             high=0.1, low=-0.1)
        
        self.activation_fn = get_activation_fn(p_activ_fn_str)
        
#-------------------------------------------------------------
def model__create():
    layers_lst = []
    for l in layers_specs_lst:
        
        size_M, size_N, activ_fn_str = l
        
        layer = Layer(size_M, size_N, activ_fn_str)
        layers_lst.append(layer)
    model = Model(layers_lst)
    return model

#-------------------------------------------------------------
def model__forward(p_x, p_model):
    
    L_prev = p_x
    for l in p_model.layers_lst:
        
        print("shapes...")
        print("W   shape - %s"%(str(l.W.shape)))
        print("L-1 shape - %s"%(str(L_prev.shape)))
        print("b   shape - %s"%(str(l.b.shape)))
        
        # print(l.W)
        # print(L_prev)
        
        # z = W * x + b
        dot = np.dot(L_prev, l.W)
        Z   = dot + l.b
        a   = l.activation_fn(Z)
        
        print("a   shape - %s"%(str(a.shape)))
        
        L_prev = np.transpose(a)
        print("L-1 shape - %s"%(str(L_prev.shape)))
        
    y = L_prev
    return y

#-------------------------------------------------------------
layers_specs_lst = [
    (1, 64,  "sigmoid"),
    (64, 10, "sigmoid"),
    (10, 1,  "softmax")
]
model = model__create()


x = X_test[0]

# IMPORTANT!! - so that the image even though 1D array of 64pixels
#               still comes in as a 2D numpy array 1x64. 
#               this is important to simplify code in forward pass
#               (L_prev = np.transpose(a))
x_input = x.reshape(x.shape[0], 1)

print(x.reshape(1, x.shape[0]).shape)



print("forward pass...")
y = model__forward(x_input, model)


print(y)


(1, 64)
forward pass...
shapes...
W   shape - (1, 64)
L-1 shape - (64, 1)
b   shape - (64,)
a   shape - (64, 64)
L-1 shape - (64, 64)
shapes...
W   shape - (64, 10)
L-1 shape - (64, 64)
b   shape - (10,)
a   shape - (64, 10)
L-1 shape - (10, 64)
shapes...
W   shape - (10, 1)
L-1 shape - (10, 64)
b   shape - (1,)


ValueError: shapes (10,64) and (10,1) not aligned: 64 (dim 1) != 10 (dim 0)